In [36]:
!pip install -U mplsoccer

^C


In [22]:
import json
import pandas as pd
import openpyxl
import os
import glob
import numpy as np
import matplotlib.pyplot as plt
from mplsoccer import Pitch, VerticalPitch, FontManager

In [23]:
def draw_pitch(ax):

    pitch_width = 80    
    pitch_length = 120  
    center_circle_radius = 9.15 
    penalty_area_length = 18  
    penalty_area_width = 44   
    goal_area_length = 6      
    goal_area_width = 20     


    ax.plot([0, 0], [0, pitch_length], color='black') 
    ax.plot([pitch_width, pitch_width], [0, pitch_length], color='black')  
    ax.plot([0, pitch_width], [0, 0], color='black')  
    ax.plot([0, pitch_width], [pitch_length, pitch_length], color='black')  

    ax.plot([0, pitch_width], [pitch_length / 2, pitch_length / 2], color='black')

    center_circle = plt.Circle((pitch_width / 2, pitch_length / 2), center_circle_radius, color='black', fill=False)
    ax.add_artist(center_circle)


    ax.plot([(pitch_width - penalty_area_width) / 2, (pitch_width - penalty_area_width) / 2],
            [0, penalty_area_length], color='black')
    ax.plot([(pitch_width + penalty_area_width) / 2, (pitch_width + penalty_area_width) / 2],
            [0, penalty_area_length], color='black')
    ax.plot([(pitch_width - penalty_area_width) / 2, (pitch_width + penalty_area_width) / 2],
            [penalty_area_length, penalty_area_length], color='black')

    ax.plot([(pitch_width - penalty_area_width) / 2, (pitch_width - penalty_area_width) / 2],
            [pitch_length - penalty_area_length, pitch_length], color='black')
    ax.plot([(pitch_width + penalty_area_width) / 2, (pitch_width + penalty_area_width) / 2],
            [pitch_length - penalty_area_length, pitch_length], color='black')
    ax.plot([(pitch_width - penalty_area_width) / 2, (pitch_width + penalty_area_width) / 2],
            [pitch_length - penalty_area_length, pitch_length - penalty_area_length], color='black')

    ax.plot([(pitch_width - goal_area_width) / 2, (pitch_width - goal_area_width) / 2],
            [0, goal_area_length], color='black')
    ax.plot([(pitch_width + goal_area_width) / 2, (pitch_width + goal_area_width) / 2],
            [0, goal_area_length], color='black')
    ax.plot([(pitch_width - goal_area_width) / 2, (pitch_width + goal_area_width) / 2],
            [goal_area_length, goal_area_length], color='black')

    ax.plot([(pitch_width - goal_area_width) / 2, (pitch_width - goal_area_width) / 2],
            [pitch_length - goal_area_length, pitch_length], color='black')
    ax.plot([(pitch_width + goal_area_width) / 2, (pitch_width + goal_area_width) / 2],
            [pitch_length - goal_area_length, pitch_length], color='black')
    ax.plot([(pitch_width - goal_area_width) / 2, (pitch_width + goal_area_width) / 2],
            [pitch_length - goal_area_length, pitch_length - goal_area_length], color='black')


    penalty_spot_distance = 12 


    ax.scatter(pitch_width / 2, penalty_spot_distance, color='black')


    ax.scatter(pitch_width / 2, pitch_length - penalty_spot_distance, color='black')

 
    ax.set_xlim(0, pitch_width)
    ax.set_ylim(0, pitch_length)
    ax.set_aspect('equal')
    ax.axis('off')


In [28]:
# D:/SoccerMatchData/data/events
folder_path = r"C:\Users\bouza\OneDrive\Desktop\data\events"
output_path = 'D:/SoccerMatchData/data/try'
goal = []
non_goal = []
json_files = glob.glob(f'{folder_path}/*.json')

for json_file in json_files:
    flag = True
    with open(json_file, 'r', encoding='utf-8') as f:
        data = json.load(f)
        # lineup_data = []
        if (data[0]['type']['name'] == 'Starting XI' and data[0]['team']['name'] == 'Barcelona') or data[1]['type']['name'] == 'Starting XI' and data[1]['team']['name'] == 'Barcelona':
            flag = False

        
    game_data = []
    i = 0
    if flag:
        continue    
    for match in data:
        if match['type']['name'] == "Shot" and match['team']['name'] == 'Barcelona':
            game_data.append({
                'Time': match['timestamp'],
                'Team': match['team']['name'],
                'Player': match['player']['name'],
                'Location': match['location'],
                'Outcome': match['shot']['outcome']['name'],
                'Technique': match['shot']['technique']['name'],
                'Body_part': match['shot']['body_part']['name']
            })

    df = pd.DataFrame(game_data)

    #print(df[df['Outcome'] == 'Goal'])
    df_goals = (df[df['Outcome'] == 'Goal'])
    df_non_goal = (df[df['Outcome'] != 'Goal'])
    for index, row in df_goals.iterrows():
        location = row['Location']
        goal.append(location)
    for index, row in df_non_goal.iterrows():
        location = row['Location']
        non_goal.append(location)
    
    
goal_coor = np.array(goal)
non_goal_coor = np.array(non_goal)
# pitch = VerticalPitch(pad_bottom=0.5,  
#                       half=True,  
#                       goal_type='box',
#                       goal_alpha=0.8, pitch_color='#22312b', line_color='#c7d5cc')  # control the goal transparency

# fig, ax = pitch.draw(figsize=(12, 10))

# ax.scatter(goal_coor[:, 1], goal_coor[:, 0], c='#ad993c', s = 5, label='Goal')

# ax.scatter(non_goal_coor[:, 1], non_goal_coor[:, 0], c='#ba4f45', s = 5, label='No goal')

# ax.legend()
# plt.show()

,Time,Team,Player,Location,Outcome,Technique,Body_part
0,00:09:37.349,Barcelona,Andrés Iniesta Luján,"[98.5, 25.5]",Off T,Normal,Right Foot
1,00:11:29.077,Barcelona,Jordi Alba Ramos,"[106.9, 26.1]",Blocked,Half Volley,Left Foot
2,00:16:48.437,Barcelona,Luis Alberto Suárez Díaz,"[96.8, 25.7]",Blocked,Normal,Right Foot
3,00:17:47.753,Barcelona,Gerard Piqué Bernabéu,"[108.5, 39.4]",Saved,Half Volley,Right Foot
4,00:24:46.176,Barcelona,Ivan Rakitić,"[109.4, 39.7]",Off T,Normal,Head
5,00:26:37.104,Barcelona,Luis Alberto Suárez Díaz,"[108.2, 48.9]",Off T,Normal,Right Foot
6,00:11:21.786,Barcelona,Philippe Coutinho Correia,"[97.3, 26.4]",Goal,Normal,Right Foot
7,00:19:45.762,Barcelona,Ivan Rakitić,"[101.5, 56.7]",Off T,Normal,Left Foot
8,00:22:18.319,Barcelona,Gerard Piqué Bernabéu,"[109.4, 42.4]",Saved,Volley,Right Foot
9,00:23:28.518,Barcelona,Denis Suárez Fernández,"[109.3, 50.4]",Off T,Normal,Right Foot


In [37]:
left_post_team1 = np.array([0, 36]) 
right_post_team1 =  np.array([0, 44])  
left_post_team2 = np.array([120, 44]) 
right_post_team2 = np.array([120, 36])
  
def perceivedLength(shot_location):
    global left_post_team1, right_post_team2, left_post_team2, right_post_team1
    first_dist = np.linalg.norm(shot_location - left_post_team1)
    second_dist = np.linalg.norm(shot_location - right_post_team2)
    # print("First distance is " + str(first_dist))
    # print("Second distance is " + str(second_dist))
    if first_dist < second_dist: 
        left_post = left_post_team1
        right_post = right_post_team1
    else:  
        left_post = left_post_team2
        right_post = right_post_team2
    # print("Left post loc is " + str(left_post))
    # print("Right post loc is " + str(right_post))
    post_to_post_dist = np.linalg.norm(left_post - right_post) 
    shot_to_left_post = np.linalg.norm(shot_location - left_post)  
    shot_to_right_post = np.linalg.norm(shot_location - right_post) 
    # print("Shot loc is " + str(shot_location))
    # print("Post to post "  + str(post_to_post_dist))
    # print("Post to left " + str(shot_to_left_post))
    # print("Post to right " + str(shot_to_right_post))
    cos_theta = (shot_to_left_post**2 + shot_to_right_post**2 - post_to_post_dist**2) / (2 * shot_to_left_post * shot_to_right_post)
    angle_rad = np.arccos(cos_theta)  
    
    angle_deg = np.degrees(angle_rad)
    return angle_deg

test = np.array([18,36])

print(perceivedLength(test))

23.962488974578164


In [43]:
df['angle'] = df['Location'].apply(lambda loc: perceivedLength(np.array(loc)))
print(df)

            Time       Team                          Player       Location  \
0   00:09:37.349  Barcelona            Andrés Iniesta Luján   [98.5, 25.5]   
1   00:11:29.077  Barcelona                Jordi Alba Ramos  [106.9, 26.1]   
2   00:16:48.437  Barcelona        Luis Alberto Suárez Díaz   [96.8, 25.7]   
3   00:17:47.753  Barcelona           Gerard Piqué Bernabéu  [108.5, 39.4]   
4   00:24:46.176  Barcelona                    Ivan Rakitić  [109.4, 39.7]   
5   00:26:37.104  Barcelona        Luis Alberto Suárez Díaz  [108.2, 48.9]   
6   00:11:21.786  Barcelona       Philippe Coutinho Correia   [97.3, 26.4]   
7   00:19:45.762  Barcelona                    Ivan Rakitić  [101.5, 56.7]   
8   00:22:18.319  Barcelona           Gerard Piqué Bernabéu  [109.4, 42.4]   
9   00:23:28.518  Barcelona          Denis Suárez Fernández  [109.3, 50.4]   
10  00:31:17.003  Barcelona  Lionel Andrés Messi Cuccittini  [110.2, 36.3]   
11  00:39:52.201  Barcelona        Francisco Alcácer García  [10